In [ ]:
## 가중치, ks와 앤더슨달링의 차이정도?
## 수학적인 내용은 추가적인 논문을 학습할 필요가 있음
## 발전시키는 모델이지 ----> 베이지안적 개념을 도입해 다른 방향으로 더 공부해볼 필요도?
## XGBOOST
## 파라미터... learning_rate : 얼마나 빠르게 학습할지, 가중치를 얼마나 높일지..
## n_estimator : 가중치 업데이트를 몇번이나 할건지로 생각할 수 있지.. 많으면 오버피팅되니까 적절한 정도를 찾아야겠지

In [38]:
import os
import pandas as pd

os.chdir(r'C:\Users\pc\Documents\data')
data = pd.read_csv("bank-additional-full.csv", sep=';')
## yes, no를 숫자로 변환


In [39]:
data['y']=data['y'].replace({'yes': 1, 'no' : 0})

In [40]:
data['y']

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, Length: 41188, dtype: int64

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [42]:

data = pd.get_dummies(data, columns=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome'])

data['id']= range(len(data))

len(data)


train = data.sample(30000, replace=False, random_state=2021).reset_index().drop(['index'],axis=1)
test = data.loc[ ~data['id'].isin(train['id'])].reset_index().drop(['index'],axis=1)


input_var=['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']


In [43]:
## XGBOOST 설치
!pip install xgboost

In [44]:
from xgboost import XGBClassifier

In [45]:
xgb = XGBClassifier(n_estimators = 300, learning_rate=0.1)

In [46]:
train['y']

0        0
1        0
2        0
3        0
4        0
        ..
29995    0
29996    1
29997    0
29998    0
29999    0
Name: y, Length: 30000, dtype: int64

In [47]:
xgb.fit(train[input_var],train['y'])

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:26:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=8, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [8]:
prediction = xgb.predict(test[input_var])

In [9]:
test['pred'] = prediction

In [10]:
(test['pred']==test['y']).mean()

0.9133893457275652

In [11]:
for n in [100,200,300,400,500,600,700,800,900]:
    xgb = XGBClassifier(n_estimators = n , learning_rate=0.1)

    xgb.fit(train[input_var],train['y'])

    prediction = xgb.predict(test[input_var])

    test['pred'] = prediction

    print((test['pred']==test['y']).mean())

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:05:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9143725420092956
[23:05:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9139256346085091
[23:05:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9133893457275652
[23:05:32] WA

In [ ]:
### 결론을 어떻게 해석할래??
## 모델의 해석이 필요한 이유, 그리고 모델이란 구체적으로 어떻게 말할 수 있는지..
## blackbox를 어떻게...
## 특히나 금융데이터 분석의 경우 해석을 못하면 어떻게.. 치명적일 수 도 있는건데..
## 도메인특성마다 어느정도로 보수적인지, 그 해석이 얼마나 필요할지, 다양하게 생각해볼 필요가 있다


In [ ]:
##변수중요도!! 굉장히 기본적이고 고전적이지만 여기부터 알아보자  -> 트리형 모델에서 보통 쓴다.
## input중 어떤것이 target에 가장 영향을 미쳤나?? 교호작용이 있다면??
## 그 변수가 얼마나 쓰이냐?, 복잡도를 얼마나 감소시켜주냐? 두가지를 모두 계산
## 랜덤포레스트나 xgboost는 각각의 tree에서 변수중요도를 계산한후 평균

In [48]:
feature_imp = xgb.feature_importances_  ##클수록 중요

imp_df =pd.DataFrame({ 'var':input_var ,
             'imp':feature_imp})

## sorting
imp_df.sort_values(['imp'], ascending=False)
##통화 오래했니?? 가 가장 영향이 컸구나

,var,imp
9,nr.employed,0.415464
1,duration,0.039246
62,poutcome_success,0.033078
7,cons.conf.idx,0.031224
5,emp.var.rate,0.028194
...,...,...
30,education_illiterate,0.002969
36,default_yes,0.000000
44,contact_telephone,0.000000
41,loan_unknown,0.000000


In [ ]:
## randomforest의 변수중요도와
## xgboost의 변수중요도가 차이가 있는데
# 일반적으로 상위모델의 변수중요도가 더 낫다는게 있다. 따라서 이번 pj같은 경우는 nr.employed를 가장 중요한 변수로 보는게 낫겠다